In [2]:
import sys
sys.path.append("../../")
from Hack import load
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pathlib import Path
import numpy as np
import datetime as datetime
import statsmodels.api as sm

In [3]:
price = load.systemprice().load()
print(price.keys())

# resample to required resolution (full res is probably too noisy)
price = price.resample('180T').mean()
print(price)

Data/systemprice.csv
Index(['Settlement Period', 'System Sell Price(£/MWh)',
       'System Buy Price(£/MWh)', 'Net Imbalance Volume(MWh)'],
      dtype='object')
                     Settlement Period  System Sell Price(£/MWh)  \
Date                                                               
2019-04-08 00:00:00                3.0                 47.832000   
2019-04-08 03:00:00                8.5                 51.805000   
2019-04-08 06:00:00               14.5                 52.390000   
2019-04-08 09:00:00               20.5                 65.701667   
2019-04-08 12:00:00               26.5                 42.498333   
...                                ...                       ...   
2021-12-15 12:00:00               26.5                320.000000   
2021-12-15 15:00:00               32.5                206.166667   
2021-12-15 18:00:00               38.5                300.191667   
2021-12-15 21:00:00               44.5                242.350000   
2021-12-16 00:00:00  

In [4]:
# plot data
fig, axs = plt.subplots(1,1)
axs.plot(price.index, price['System Buy Price(£/MWh)'].values)

# Model

In [5]:
from statsmodels.tsa.stattools import adfuller
def adf_test(timeseries):
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

print(adf_test(price['System Buy Price(£/MWh)']))

Results of Dickey-Fuller Test:
Test Statistic                   -4.737909
p-value                           0.000071
#Lags Used                       36.000000
Number of Observations Used    7828.000000
Critical Value (1%)              -3.431186
Critical Value (5%)              -2.861909
Critical Value (10%)             -2.566967
dtype: float64
None


In [6]:
# Figure out model params
decomposition = sm.tsa.seasonal_decompose(price['System Buy Price(£/MWh)'], model='additive')
fig = decomposition.plot()
plt.show()

system_buy = price['System Buy Price(£/MWh)']
system_buy = system_buy - system_buy.shift(8)
system_buy = system_buy.dropna()
system_buy.plot()



from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_acf(system_buy)
plt.show()
plot_pacf(system_buy)
plt.show()

/Users/Tom/opt/anaconda3/envs/ml/lib/python3.9/site-packages/statsmodels/graphics/tsaplots.py:348: FutureWarning: The default method 'yw' can produce PACF values outside of the [-1,1] interval. After 0.13, the default will change tounadjusted Yule-Walker ('ywm'). You can use this method now by setting method='ywm'.
  warnings.warn(


In [7]:
# Split into train and test data
t1 = datetime.datetime(2021, 1, 1, 0, 0, 0)   # train up to this date
t2 = datetime.datetime(2021, 1, 6, 0, 0, 0)   # predict up to this date

train_data = price['System Buy Price(£/MWh)'].loc[price.index<t1]
test_data = price['System Buy Price(£/MWh)'].loc[(price.index>=t1)&(price.index<t2)]
train_data, test_data = train_data.dropna(), test_data.dropna()

In [10]:
# define model configuration
fig, axs = plt.subplots(1,1)
axs.plot(test_data.index, test_data.values, c='black')
#my_order = (0, 0, 1)
#my_seasonal_order = (1, 1, 1, 8)

combinations = [ [0, 0, 1, 1, 1, 1, 8], [1, 0, 1, 1, 1, 1, 8], [0, 1, 1, 1, 1, 1, 8], [1, 1, 1, 1, 1, 100, 8]   ]
# define model
for i in combinations:
    my_order = (i[0], i[1], i[2])
    my_seasonal_order = (i[3], i[4], i[5], i[6])

    model = sm.tsa.statespace.SARIMAX(train_data, order=my_order, seasonal_order=my_seasonal_order)
    model_fitted = model.fit()
    predictions = model_fitted.predict(start=len(train_data), end=len(train_data) + len(test_data)-1, dynamic=False)
    axs.plot(test_data.index, predictions, label=str(i))
axs.legend()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.49421D+00    |proj g|=  8.15566D-02


 This problem is unconstrained.



At iterate    5    f=  4.43993D+00    |proj g|=  2.05333D-02

At iterate   10    f=  4.39836D+00    |proj g|=  6.91169D-03

At iterate   15    f=  4.37683D+00    |proj g|=  2.73939D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     19     25      1     0     0   2.617D-06   4.377D+00
  F =   4.3767422620815131     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.48601D+00    |proj g|=  8.28723D-02

At iterate    5    f=  4.42929D+00    |proj g|=  1.67315D-02

At iterate   10    f=  4.38142D+00    |proj g|=  2.48520D-03

At iterate   15    f=  4.38036D+00    |proj g|=  9.00255D-03

At iterate   20    f=  4.37048D+00    |proj g|=  4.47265D-02

At iterate   25    f=  4.36450D+00    |proj g|=  6.41765D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     28     43      1     0     0   

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.57111D+00    |proj g|=  7.46371D-02

At iterate    5    f=  4.42763D+00    |proj g|=  1.24936D-02

At iterate   10    f=  4.40928D+00    |proj g|=  8.40717D-03

At iterate   15    f=  4.40681D+00    |proj g|=  3.31178D-04

At iterate   20    f=  4.40679D+00    |proj g|=  1.40340D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     20     25      1     0     0   1.403D-05   4.407D+00
  F =   4.4067875231226052     

CONVERG

/Users/Tom/opt/anaconda3/envs/ml/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
